In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ''

def formatData(data):
    if data == 'no':
        return 'A multi-line chart that does not suffer from overfitting.'
    elif 'overfitting after' in data and 'acc=' not in data:
        return f'A multi-line chart that shows {data}.'
    elif 'overfitting until' in data:
        return f'A multi-line chart that shows {data} neighbors.'
    elif ',acc' in data and 'overfitting' in data:
        if 'no' in data.split("nderfitting ")[0]:
            underfitting = 'no'
        else:
            underfitting = data.split("nderfitting ")[1].split(",")[0]
        
        if 'no' in data.split("overfitting ")[0]:
            overfitting = 'no'
        else:
            overfitting = data.split("overfitting ")[1].split(",")[0]
         
        if data.split("ac")[1] == 'c>80':
            acc = 'test accuracy line surpasses than 80%'
        elif data.split("ac")[1] == 'c<80':
            acc = 'test accuracy line does not surpass 80%'
        else:
            acc = 'test accuracy line stays at 80%'
        if underfitting == 'no' and overfitting == 'no':
            return f'A multi-line chart showing the random forest does not have underfitting, does not enter in overfitting and it\'s {acc}.'
        elif underfitting != 'no' and overfitting == 'no':
            return f'A multi-line chart showing the random forest is in underfitting until {underfitting} estimators, does not enter in overfitting and it\'s {acc}.'
        elif underfitting == 'no' and overfitting != 'no':
            return f'A multi-line chart showing the random forest does not have underfitting, enters in overfitting after {overfitting} estimators and it\'s {acc}.'
        else:
            return f'A multi-line chart showing the random forest is in underfitting until {underfitting} estimators, enters in overfitting after {overfitting} estimators and it\'s {acc}.'
    elif data == 'acc_rec':
        return 'A multi-line chart showing the train and test accuracy and recall of a decision tree.'
    elif 'samples' in data:
        return f'A decision tree with the following configuration:{data}.'
    elif data[0] == '[' and len(data.split(",")) > 2:
        return f'A bar chart showing the explained variance ration of each principal components the values are the following:{data}.'
    elif data[0] == '{' and data[1].isnumeric():
        return f'A correlation heatmap with {data[1:].split(",")[0]} features where the redundant pairs are the following:{data[:-1].split(",",1)[1]}.'
    elif data[0] == '[' and (data[1] == 'T' or data[1] == 'F') and data[2] == ',':
        return f'A set of boxplots that aren\'t normalized, in the following dictionary we can see each one of them and see if they have outliers and are balanced:{data[:-1].split(",",1)[1]}.'
    elif data[-1] == '}' and data[-2] == ']':
        return f'A set of bar charts for each symbolic variable:{data}.'
    elif data[0] == '{' and data[-2].isnumeric():
        return f'A bar chart showing the number of missing values for each variable:{data}.'
    elif data[0] == '[' and len(data.split(",")) == 2:
        return f'A bar chart with the following values:{data}.'
    elif 'Ordinal:' in data:
        return f'A set of histograms one for each variable, in the following dictionary we can see each one of them and see if they have outliers, if they are balanced and if it makes sense for them to be ordinal:{data}.'


In [13]:
from PIL import Image
import requests
import torch
from datasets import load_dataset
from transformers import AutoProcessor
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"

filetag = 'final-model-special'

dataset = load_dataset('eduvedras/QA',split='test',trust_remote_code=True)

In [ ]:
checkpoint = "eduvedras/ChartDataModel_Pix2Struct"

processor = Pix2StructProcessor.from_pretrained("google/pix2struct-textcaps-base")

In [15]:
from huggingface_hub import InferenceClient
from transformers import AutoModelForCausalLM

model = Pix2StructForConditionalGeneration.from_pretrained(checkpoint).to(device)
model

Pix2StructForConditionalGeneration(
  (encoder): Pix2StructVisionModel(
    (embeddings): Pix2StructVisionEmbeddings(
      (patch_projection): Linear(in_features=768, out_features=768, bias=True)
      (row_embedder): Embedding(4096, 768)
      (column_embedder): Embedding(4096, 768)
      (dropout): Dropout(p=0.06, inplace=False)
    )
    (encoder): Pix2StructVisionEncoder(
      (layer): ModuleList(
        (0-11): 12 x Pix2StructVisionLayer(
          (attention): Pix2StructVisionAttention(
            (query): Linear(in_features=768, out_features=768, bias=False)
            (key): Linear(in_features=768, out_features=768, bias=False)
            (value): Linear(in_features=768, out_features=768, bias=False)
            (output): Linear(in_features=768, out_features=768, bias=False)
          )
          (mlp): Pix2StructVisionMlp(
            (wi_0): Linear(in_features=768, out_features=2048, bias=False)
            (wi_1): Linear(in_features=768, out_features=2048, bias=False)


In [16]:
from tqdm.auto import tqdm
import pandas as pd
from openai import OpenAI

client = OpenAI()

special = ['decision_tree','overfitting_dt_acc_rec','pca','mv','class_histogram','nr_records_nr_variables']

def isSpecial(chart):
    for i in special:
        if i in chart:
            return True
    
    return False

predictions = []
references = []
questions = []
for i in tqdm(range(len(dataset))):
    if 'overfitting_decision_tree' in dataset[i]['Chart_name'] or not isSpecial(dataset[i]['Chart_name']):
        continue
    inputs = processor(images=dataset[i]['Chart'], return_tensors="pt").to(device)
    flattened_patches = inputs.flattened_patches
    attention_mask = inputs.attention_mask
    generated_ids = model.generate(flattened_patches=flattened_patches, attention_mask=attention_mask, max_length=516)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    chartdata = formatData(generated_caption)
    question = dataset[i]['Question']
        
    inputs = []
    inputs.append({"role": "system", "content": "You are a student doing a data science exam."})
    inputs.append({"role":"user", "content":f"Consider the following chart \"{chartdata}\", classify the following sentence about the chart as true or false: \"{question}\"."})

    print(chartdata)
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal::9YKGQaaF",
        messages=inputs
    )
    print(response.choices[0].message.content)
    predictions.append(response.choices[0].message.content)
    references.append(dataset[i]["Answer"])
    questions.append(question)
    

  0%|          | 0/323 [00:00<?, ?it/s]

A decision tree with the following configuration:{'Int_memory <= 30.5': {'samples': 350, 'value': [350,350,350,350], 'class': '0', 'True': {'mobile_wt <= 678': {'samples': 678, 'value': [179,169,182,148], 'class': '1', 'True': {'samples': 460, 'value': [114,128,118,109], 'class': '1'}, 'False': {'samples': 209, 'value': [65,47,34,39], 'class': '0'}}}, 'False': {'mobile_wt <= 91.5': {'samples': 394, 'value': [171,181,168,202], 'class': '3', 'True': {'samples': 86, 'value': [26,19,11,30], 'class': '3'}}}}}.


  0%|          | 1/323 [00:16<1:28:27, 16.48s/it]

True
A decision tree with the following configuration:{'Int_memory <= 30.5': {'samples': 350, 'value': [350,350,350,350], 'class': '0', 'True': {'mobile_wt <= 678': {'samples': 678, 'value': [179,169,182,148], 'class': '1', 'True': {'samples': 460, 'value': [114,128,118,109], 'class': '1'}, 'False': {'samples': 209, 'value': [65,47,34,39], 'class': '0'}}}, 'False': {'mobile_wt <= 91.5': {'samples': 394, 'value': [171,181,168,202], 'class': '3', 'True': {'samples': 86, 'value': [26,19,11,30], 'class': '3'}}}}}.


  1%|          | 2/323 [00:32<1:25:35, 16.00s/it]

...

 93%|█████████▎| 302/323 [10:19<00:05,  3.65it/s]

True
A bar chart with the following values:[4653,9].
False
A bar chart with the following values:[4653,9].


 95%|█████████▌| 307/323 [10:21<00:04,  3.42it/s]

False


100%|██████████| 323/323 [10:22<00:00,  1.93s/it]


In [20]:
import pandas as pd

new_df = pd.DataFrame(columns=['Image','Question','Prediction','Reference'])
j=0
for i in tqdm(range(len(dataset))):
    if 'overfitting_decision_tree' in dataset[i]['Chart_name'] or not isSpecial(dataset[i]['Chart_name']):
        continue
    if predictions[j] == 'True':
        pred = 'T'
    elif predictions[j] == 'False':
        pred = 'F'
    else:
        pred = '?'
    new_df.loc[len(new_df)] = {'Image': dataset[i]['Chart_name'], 'Question': questions[j], 'Prediction': pred, 'Reference': references[j]}
    j+=1
    
new_df.to_csv(f'./results/{filetag}.csv', sep=";", index=False)

KeyboardInterrupt: 